<a href="https://colab.research.google.com/github/AnusreeChittineni/VAIL_2021/blob/main/Sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# unzips neccessarry zip files in Drive
# only needs to be run once for your Drive
!unzip -uq "/content/drive/MyDrive/Sarcasm_Headlines_Dataset.json.zip" -d "/content/drive/My Drive/SarcasmDetection"

In [50]:
import json

# reading data
def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

# intializes a list called data to store data
data = list(parse_data('/content/drive/MyDrive/SarcasmDetection/Sarcasm_Headlines_Dataset.json'))
print(data[0])

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5', 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers", 'is_sarcastic': 0}


In [67]:
import pandas as pd

df = pd.DataFrame(data)
print(df.head())

                                        article_link  ... is_sarcastic
0  https://www.huffingtonpost.com/entry/versace-b...  ...            0
1  https://www.huffingtonpost.com/entry/roseanne-...  ...            0
2  https://local.theonion.com/mom-starting-to-fea...  ...            1
3  https://politics.theonion.com/boehner-just-wan...  ...            1
4  https://www.huffingtonpost.com/entry/jk-rowlin...  ...            0

[5 rows x 3 columns]


In [143]:
from sklearn.model_selection import train_test_split
import sklearn
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import numpy as np

In [124]:
# initlaizes input and output values
x = df['headline'].values
y = df['is_sarcastic'].values

# splits data into training and testing groups
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

# prints shape of data
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(21367,) (5342,) (21367,) (5342,)


In [125]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(x_train) 

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(x_train)
max_sequence = max(len(x) for x in training_sequences)
training_padded = np.array([np.pad(x, (0, max_sequence - len(x))) for x in training_sequences])

In [126]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(len(word_index) + 1, 2, input_shape=[None]),
        tf.keras.layers.LSTM(2, return_sequences=True),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     (None, None, 2)           53136     
_________________________________________________________________
lstm_25 (LSTM)               (None, None, 2)           40        
_________________________________________________________________
dense_50 (Dense)             (None, None, 1)           3         
Total params: 53,179
Trainable params: 53,179
Non-trainable params: 0
_________________________________________________________________


In [130]:
# try with various optimizer and loss algorithms
# (SGD, MSE, 56), (Adam, MSE, 93), (Adam, BC, 97)
model.compile(optimizer='Adam', loss='BinaryCrossentropy', metrics=['accuracy'])

model.fit(training_padded, y_train, 
          epochs=5, verbose=1)

Epoch 1/5
668/668 [==============================] - 15s 19ms/step - loss: 0.1909 - accuracy: 0.9475
Epoch 2/5
668/668 [==============================] - 14s 20ms/step - loss: 0.1630 - accuracy: 0.9558
Epoch 3/5
668/668 [==============================] - 14s 20ms/step - loss: 0.1372 - accuracy: 0.9632
Epoch 4/5
668/668 [==============================] - 14s 20ms/step - loss: 0.1243 - accuracy: 0.9661
Epoch 5/5
668/668 [==============================] - 14s 20ms/step - loss: 0.1025 - accuracy: 0.9720


In [152]:
testing_sequences = tokenizer.texts_to_sequences(x_test)
y_test = np.array(y_test)

In [155]:
results = model.evaluate(x_test, y_test)
print("test loss, test acc:", results)

predictions = model.predict(np.array([testing_sequences]))

print(sklearn.metrics.accuracy_score(predictions, y_test))

UnimplementedError: ignored